<a href="https://colab.research.google.com/github/ludwigwittgenstein2/Research/blob/master/Hugging_Face_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 3.0MB 16.7MB/s 
     |████████████████████████████████| 1.1MB 37.5MB/s 
     |████████████████████████████████| 890kB 32.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=f610b191f8e1fbe71fbf25f866fbfb726bb1acc34a6b7db65206e80011124ee0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [5]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

In [11]:
classifier('We are very happy to show you the 😜 Transfomers Library')


[{'label': 'POSITIVE', 'score': 0.9998140335083008}]

In [12]:
results = classifier(["We are very happy to show you the 🤫 Transfomers library", "we hope you don't hate it"])

for result in results: 
  print(f"label:{result['label']}, with score:{round(result['score'], 4)}")

label:POSITIVE, with score:0.9998
label:POSITIVE, with score:0.832


In [13]:
classifier = pipeline('sentiment_')

[{'label': 'POSITIVE', 'score': 0.9998140335083008},
 {'label': 'POSITIVE', 'score': 0.8319633603096008}]